In [1]:
# train_model.py
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import joblib
import re

# 1. Veriyi Yükle
df = pd.read_csv("netflixData.csv")

# 2. Veri Temizleme ve Ön İşleme
# Eksik verileri doldur
df['Description'] = df['Description'].fillna('')
df['Genres'] = df['Genres'].fillna('')
df['Director'] = df['Director'].fillna('')
df['Cast'] = df['Cast'].fillna('')

# Metin temizleme fonksiyonu
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Tüm önemli özellikleri birleştirip tek bir "Soup" (Çorba) oluşturuyoruz
# Bu yöntem, hem konuya hem türe hem de oyuncuya göre benzerlik bulmamızı sağlar.
df['combined_features'] = (
    df['Description'] + " " + 
    df['Genres'] + " " + 
    df['Director'] + " " + 
    df['Cast']
)

df['combined_features'] = df['combined_features'].apply(clean_text)

# 3. TF-IDF Vektörleştirme
# İngilizce stop words (the, a, and vb.) çıkarılır
tfidf = TfidfVectorizer(stop_words='english', max_features=5000) 
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# 4. Kosinüs Benzerliği (Cosine Similarity) Hesaplama
# linear_kernel, cosine_similarity ile aynı işi yapar ama daha hızlıdır.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 5. Kaydetme
# Veri seti büyükse cosine_sim matrisi çok yer kaplayabilir.
# Bu yüzden sadece gerekli dosyaları kaydediyoruz.
# Not: Tüm matrisi kaydetmek yerine uygulamada canlı hesaplamak (küçük veri için)
# veya sadece en iyi 10 benzerliği kaydetmek daha verimli olabilir.
# Ancak bu veri seti (6000 satır) için matrisi kaydetmek sorun olmaz.

# İndeksleri kolay bulmak için bir harita oluştur
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

joblib.dump(cosine_sim, "netflix_cosine_sim.pkl")
joblib.dump(indices, "netflix_indices.pkl")
df.to_csv("netflix_processed.csv", index=False)

print("Model (Benzerlik Matrisi) ve işlenmiş veri kaydedildi.")

Model (Benzerlik Matrisi) ve işlenmiş veri kaydedildi.
